In [ ]:
import os
from PIL import Image
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fft import fft, ifft
from scipy import fftpack
from scipy import signal
import soundfile as sf
from os.path import basename
from skimage import io
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [ ]:
audio_directory = 'dataset_renomme/Covid/audio/'
SIZE = 500
dataset = {}

my_audios = os.listdir(audio_directory)
for i, audio_name in enumerate(my_audios):
    if (audio_name.split('.')[1] == 'wav'):
            audio,sr = librosa.load(audio_directory+audio_name)
            dataset[audio_name] = audio

In [ ]:
def plot_spectrogram(Y, sr, hop_length, y_axis="linear"):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y, 
                             sr=sr, 
                             hop_length=hop_length, 
                             x_axis="time", 
                             y_axis=y_axis)
    plt.colorbar(format="%+2.f")

In [ ]:
def transformation(name,scale):
# création des variables Fourier et Fréquences, qui permettent de construire le spectre du signal.
    fourier = fftpack.fft(scale)
    power = np.abs(fourier)
# la variable power est créée pour éiminer les amplitudes négatives
    frequences = fftpack.fftfreq(scale.size)

# filtre du spectre avec du boolean indexing de Numpy
    fourier[ power < np.percentile(power,90) ] = 0
# Transformation de Fourier Inverse: genere un nouveau signal temporel depuis le spectre filtré
    filtered_signal = fftpack.ifft(fourier)

    FRAME_SIZE = 2048
    HOP_SIZE = 512

    S_scale = librosa.stft(np.abs(filtered_signal), n_fft=FRAME_SIZE, hop_length=HOP_SIZE) #librosa.stft(scale, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
    Y_scale = np.abs(S_scale) ** 2

    Y_log_scale = librosa.power_to_db(Y_scale)

    plot_spectrogram(Y_log_scale, sr, HOP_SIZE, y_axis="log")
    fileName1, fileExtension = os.path.splitext(name)
    fileName1 = str(fileName1) + ".png"
    plt.savefig('dataset_renomme/Covid/spectrogram'+fileName1)

In [ ]:
for cle,valeur in dataset.items():
    transformation(cle,valeur)